# 矩阵分解

In [1]:
import numpy as np
import scipy
from scipy import linalg
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display,Latex,Math
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
sh = InteractiveShell.instance()

def number_to_str(n,cut=5):
    ns=str(n)
    format_='{0:.'+str(cut)+'f}'
    if 'e' in ns or ('.' in ns and len(ns)>cut+1):
        return format_.format(n)
    else:
        return str(n)

def matrix_to_latex(mat,style='bmatrix'):
    if type(mat)==np.matrixlib.defmatrix.matrix:
        mat=mat.A
    head=r'\begin{'+style+'}'
    tail=r'\end{'+style+'}'
    if len(mat.shape)==1:
        body=r'\\'.join([str(el) for el in mat])
        return head+body+tail
    elif len(mat.shape)==2:
        lines=[]
        for row in mat:
            lines.append('&'.join([number_to_str(el)  for el in row])+r'\\')
        s=head+' '.join(lines)+tail
        return s
    return None

sh.display_formatter.formatters['text/latex'].type_printers[np.ndarray]=matrix_to_latex

def show_decomposition(*args):
    latex=''
    for arg in args:
        if type(arg)==str:
            latex+=arg
        else:
            latex+=matrix_to_latex(arg)
    latex='$'+latex+'$'
    display(Math(latex))


## 三角分解

三角分解就是把矩阵分解成上下三角的乘积形式，这样就可以利用回代法直接得到解。

### 回代法

考虑形式为

$$
\begin{bmatrix}
 1 & 2 & 3 \\
 0 & 4 & 5 \\
 0 & 0 & 6 \\
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2 \\
x_3 \\
\end{bmatrix}
=
\begin{bmatrix}
7 \\
8 \\
9 \\
\end{bmatrix}\\
$$
的方程，显然可以从右下网上机械的求解

In [2]:
def subsup(A,b):
    size=A.shape[1]
    x=np.zeros(size)
    for i in range(size-1,-1,-1):
        x[i]=(b[i]-sum([x[j]*A[i][j] for j in range(i+1,size)]))/A[i][i]   
    return x



In [3]:
A=np.array([1,2,3,0,4,5,0,0,6]).reshape(3,3)
b=np.array([7,8,9])

x=subsup(A,b)
x

array([ 2.25 ,  0.125,  1.5  ])

In [4]:
show_decomposition(A,x,'=',A.dot(x))

<IPython.core.display.Math object>

下三角的情况类似
$$
\begin{bmatrix}
 1 & 0 & 0 \\
 2 & 3 & 0 \\
 4 & 5 & 6 \\
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2 \\
x_3 \\
\end{bmatrix}
=
\begin{bmatrix}
7 \\
8 \\
9 \\
\end{bmatrix}\\
$$


In [5]:
def subsdown(A,b):
    size=A.shape[1]
    x=np.zeros(size)
    for i in range(size):
        x[i]=(b[i]-sum([A[i][j]*x[j] for j in range(i)]))/A[i][i]
    return x

In [6]:
A=np.array([1,0,0,2,3,0,4,5,6]).reshape(3,3)
b=np.array([7,8,9])
x=subsdown(A,b)
x

array([ 7. , -2. , -1.5])

In [7]:
show_decomposition(A,x,'=',A.dot(x))

<IPython.core.display.Math object>

从而我们也就见识了上下三角矩阵的回代如何容易解决这种方成求解问题。那么显然，对于一般的

$$
Ax=b
$$

问题，我们如果能将$A$分解成下三角矩阵$L$与上三角矩阵$R$。则我们换元来利用回代法解决问题。

$$
LRx=b \\
Rx=y \\
Ly=b \\
$$

于是可以先利用下三角矩阵回代法求出$y$，再由$Rx=y$求出$x$。这种方法不知为何有种似曾相识的感觉...

### LR分解

LR分解还可以要求L的对角线上元全为1.

$$
\begin{bmatrix}
a_{11} & a_{12} & \cdots & a_{1n} \\
a_{21} & a_{22} & \cdots & a_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
a_{n1} & a_{n2} & \cdots & a_{nn} \\
\end{bmatrix}
=
\begin{bmatrix}
1 & 0 & \cdots & 0 \\
l_{21} & 1 & \cdots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
l_{n1} & l_{n2} & \cdots & 1 \\
\end{bmatrix}
\begin{bmatrix}
r_{11} & r_{12} & \cdots & r_{1n} \\
0 & r_{22} & \cdots & r_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \cdots & r_{nn} \\
\end{bmatrix}
$$

In [29]:
def lr(A):
    size=A.shape[0]
    L=np.diag(np.ones(size))
    R=np.zeros(A.shape)
    for t in range(size):
        R[0][t]=A[0][t]
    for l in range(1,size):
        for i in range(l):
            L[l][i]=(A[l][i]-sum([L[l][jj]*R[jj][i] for jj in range(i) ]))/R[i][i]
        for j in range(l,size):
            R[l][j]=A[l][j]-sum([L[l][jj]*R[jj][j] for jj in range(l)])
    return L,R


            

In [33]:
A=np.array([2,3,0,0,1,0,4,6,0]).reshape(3,3)

A

array([[2, 3, 0],
       [0, 1, 0],
       [4, 6, 0]])

In [34]:
L,R=lr(A)

show_decomposition(L.dot(R),'=',L,R)

<IPython.core.display.Math object>

如此，根据上面的换元思想，就可以定义出一个基于lr分解的方程组的解法，当然在这里并没有显示出比高斯消元法任何的优势。

In [35]:
def solve(A,b):
    L,R=lr(A)
    y=subsdown(L,b)
    x=subsup(R,y)
    return x

In [52]:
A=(np.arange(1,10)**2).reshape(3,3)
A

array([[ 1,  4,  9],
       [16, 25, 36],
       [49, 64, 81]])

In [53]:
b=np.arange(1,4)
b

array([1, 2, 3])

In [38]:
np.linalg.solve(A,b)

array([ 0.08333333, -0.33333333,  0.25      ])

In [39]:
solve(A,b)

array([ 0.08333333, -0.33333333,  0.25      ])

In [50]:
L,R=lr(A)

show_decomposition(L.dot(R),'=',L,R)

<IPython.core.display.Math object>

In [54]:
solve(A,b) # LR decomposition way

array([ 0.08333333, -0.33333333,  0.25      ])